論文  
https://arxiv.org/abs/2011.09055  

Dataset  
https://svip-lab.github.io/dataset/iPER_dataset.html  

GitHub  
https://github.com/iPERDance/iPERCore  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/iPERCore_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

## GPUの確認

In [ ]:
!nvidia-smi

## ライブラリのインストール

In [ ]:
# Install ffmpeg (ffprobe)
!apt-get install ffmpeg

## ライブラリのインポート

In [ ]:
import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess

from google.colab import files
from moviepy.editor import *
from moviepy.video.fx.resize import resize

## 環境変数設定

In [ ]:
!ls /usr/local/ | grep cuda

# CUDA_HOMEにcuda-10.1のパスを設定
os.environ["CUDA_HOME"] = "/usr/local/cuda-10.1"

!echo $CUDA_HOME

# iPERCoreのセットアップ

## GitHubからコードを取得

In [ ]:
!git clone https://github.com/iPERDance/iPERCore.git

## ライブラリのインストール

In [ ]:
%cd /content/iPERCore/

!python setup.py develop

## 学習済みモデルのダウンロード

In [ ]:
# Download all checkpoints
#!wget -O assets/checkpoints.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLkwQyrk3W9_H7MzNA?e=rRje0G"
!wget -O assets/checkpoints.zip "https://download.impersonator.org/iper_plus_plus_latest_checkpoints.zip"
!unzip -o assets/checkpoints.zip -d assets/

!rm assets/checkpoints.zip

## サンプルデータのダウンロード

In [ ]:
# download samples
# !wget -O assets/samples.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLobQPpoxo2hfhURrA?e=EUyIC2"
!wget -O assets/samples.zip  "https://download.impersonator.org/iper_plus_plus_latest_samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

# Motion Imitation

## configの詳細
  --gpu_ids（str）：gpu_ids、デフォルトは「0」です  
  --image_size（int）：画像サイズ。デフォルトは512です  
  --num_source（int）：Attentionのソースイメージの数。デフォルトは2です。Largeにはより多くのGPUメモリが必要です  
  --assets_dir（str）：アセットディレクトリ。configurationとすべての事前トレーニング済みチェックポイントが必要  
  --output_dir（str）：出力ディレクトリ  
  --src_path（str）：ソース入力情報  

In [ ]:
%cd /content/iPERCore
!rm -rf images
!mkdir images


#@markdown  front_only:正面画像のみ、front_back:正面、背面画像、fron_back_bg:正面、背面、背景画像
model_type = "front_back"  #@param ["front_only", "front_back", "front_back_bg"] {allow-input: false}

gpu_ids = "0"
#@markdown 出力画像サイズ
image_size = 512 #@param {type:"integer"}
num_source = 2
assets_dir = "/content/iPERCore/assets"
output_dir = "/content/iPERCore/results"
cfg_path = "/content/iPERCore/assets/configs/deploy.toml"

# model設定
model_id = "donald_trump_2"
if model_type == "front_back":
  model_id = "axing_1"
elif model_type == "front_back_bg":
  model_id = "afan_6=ns=2"

%cd /content/iPERCore/images
# src画像アップロード
if model_type == "front_only":
  print("正面画像をアップロードしてください")
  front_img = files.upload()
  front_img = list(front_img.keys())

  front_img_full_path = os.path.join("/content/iPERCore/images", front_img[0])
  src_path = "\"path?=" + front_img_full_path + ",name?=front_only\""
elif model_type == "front_back":
  print("正面画像をアップロードしてください")
  front_img = files.upload()
  front_img = list(front_img.keys())
  print("背面画像をアップロードしてください")
  back_img = files.upload()
  back_img = list(back_img.keys())

  src_path = "\"path?=/content/iPERCore/images,name?=front_back\""
elif model_type == "front_back_bg":
  print("正面画像をアップロードしてください")
  front_img = files.upload()
  front_img = list(front_img.keys())
  print("背面画像をアップロードしてください")
  back_img = files.upload()
  back_img = list(back_img.keys())
  print("背景画像をアップロードしてください")
  bg_img = files.upload()
  bg_img = list(bg_img.keys())

  bg_img_full_path = os.path.join("/content/iPERCore/images", bg_img[0])
  src_path = "\"path?=/content/iPERCore/images," \
             "name?=front_back_bg," \
             "bg_path?=" + bg_img_full_path + "\""



In [ ]:
!mkdir videos
!rm -rf vidoos
%cd /content/iPERCore/videos
#@markdown 動画の切り抜き(秒)  end=0は変更なし
start =  47#@param {type:"integer"}
end =  60#@param {type:"integer"}

print("動きを参照する動画をアップロードしてください")
video = files.upload()
video = list(video.keys())

clip = VideoFileClip(video[0])
if end != 0:
    if start < end and end <= clip.duration:
      clip = clip.subclip(start, end)

subclip_path = os.path.join("/content/iPERCore/videos", "sub_"+video[0])
clip.write_videofile(subclip_path)
video_name = os.path.splitext(os.path.basename(video[0]))[0]

ref_path = "\"path?=" + subclip_path + "," \
             "name?=" + video_name + "," \
             "pose_fc?=400\""


In [ ]:
%%time

%cd /content/iPERCore/

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

In [ ]:
%cd /content/iPERCore/
results_video = "/content/iPERCore/results/primitives/" + model_type + "/synthesis/imitations/" + model_type + "-" + video_name + ".mp4"

clip = VideoFileClip(results_video)
clip = resize(clip, height=420)
clip.ipython_display()